In [1]:
import pdfplumber

def extract_text_with_spaces(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            page_text = ''
            words = []
            previous_char = None

            # Extract character-level details
            for char in page.chars:
                if previous_char:
                    # Calculate space based on character positions
                    space_threshold = 0.5 * (char['fontname'] == previous_char['fontname']) # Adjust this threshold if needed
                    if char['x0'] > previous_char['x1'] + space_threshold:
                        page_text += ' '
                page_text += char['text']
                previous_char = char

            text += page_text + '\n'
    return text

# Use the function to extract text with spaces
pdf_path = 'C:\\MyFolder\\earningscall.pdf'
text = extract_text_with_spaces(pdf_path)

# Print or process the text as needed
print(text)


Coinbase Global, Inc.Second Quarter 2024 Earnings CallAugust 1, 2024Anil Gupta, Vice President, Investor Relations: Good afternoon, and welcome to the Coinbase Second Quarter2024 Earnings Call. Joining me on today's call are Brian Armstrong, Co-Founder and CEO; Emilie Choi, Presidentand COO; Alesia Haas, CFO; and Paul Grewal, Chief Legal Officer.I hope you've all had the opportunity to read our shareholder letter, which was published on our InvestorRelations website earlier today. Before we get started, I'd like to remind you that during today's call we maymake forward-looking statements. Actual results may vary materially from today's statements.Information concerning risks, uncertainties and other factors that could cause these results to differ is includedin our SEC filings. Our discussion today will also include references to certain non-GAAP financial measures.Reconciliations to the most directly comparable GAAP financial measures are provided in our shareholderletter on our Inves

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter



from langchain.text_splitter import RecursiveCharacterTextSplitter

# Example transcript
transcript = text

# Create the text splitter with desired parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # Maximum size of each chunk in characters
    chunk_overlap=50,  # Overlap between chunks to preserve context
    separators=["\n\n", "\n", ".", " "]  # Order of separators to use for splitting
)

# Split the transcript into chunks
chunks = text_splitter.split_text(transcript)

# Output the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")


Chunk 1:
Coinbase Global, Inc.Second Quarter 2024 Earnings CallAugust 1, 2024Anil Gupta, Vice President, Investor Relations: Good afternoon, and welcome to the Coinbase Second Quarter2024 Earnings Call

Chunk 2:
. Joining me on today's call are Brian Armstrong, Co-Founder and CEO; Emilie Choi, Presidentand COO; Alesia Haas, CFO; and Paul Grewal, Chief Legal Officer.I hope you've all had the opportunity to read our shareholder letter, which was published on our InvestorRelations website earlier today

Chunk 3:
. Before we get started, I'd like to remind you that during today's call we maymake forward-looking statements. Actual results may vary materially from today's statements

Chunk 4:
.Information concerning risks, uncertainties and other factors that could cause these results to differ is includedin our SEC filings. Our discussion today will also include references to certain non-GAAP financial measures

Chunk 5:
.Reconciliations to the most directly comparable GAAP financial measur

In [3]:
from langchain_core.documents import Document

# Convert chunks to documents
documents = [Document(page_content=chunk, metadata={"source": "transcript"}) for chunk in chunks]


In [4]:
# Output the documents
for i, doc in enumerate(documents):
    print(doc)

page_content='Coinbase Global, Inc.Second Quarter 2024 Earnings CallAugust 1, 2024Anil Gupta, Vice President, Investor Relations: Good afternoon, and welcome to the Coinbase Second Quarter2024 Earnings Call' metadata={'source': 'transcript'}
page_content='. Joining me on today's call are Brian Armstrong, Co-Founder and CEO; Emilie Choi, Presidentand COO; Alesia Haas, CFO; and Paul Grewal, Chief Legal Officer.I hope you've all had the opportunity to read our shareholder letter, which was published on our InvestorRelations website earlier today' metadata={'source': 'transcript'}
page_content='. Before we get started, I'd like to remind you that during today's call we maymake forward-looking statements. Actual results may vary materially from today's statements' metadata={'source': 'transcript'}
page_content='.Information concerning risks, uncertainties and other factors that could cause these results to differ is includedin our SEC filings. Our discussion today will also include referenc

In [5]:
from langchain.llms import OpenAI
import chromadb
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Assuming 'chunks' is a list of strings, each representing a chunk of the transcript

# Convert chunks to the format expected by the vectorstore
# Chroma expects each document to be an object with a 'page_content' attribute.

# Initialize the embeddings
embeddings = OpenAIEmbeddings(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="nomic-ai/nomic-embed-text-v1.5-GGUF", check_embedding_ctx_length=False
)

# Initialize the ChromaDB client (using an ephemeral in-memory client for testing)
new_client = chromadb.EphemeralClient()

# Create the vectorstore and insert the documents
vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings, client=new_client)
print("Finished creating the vectorstore.")

# Initialize the retriever for use in the RAG pipeline
retriever = vectorstore.as_retriever()


Finished creating the vectorstore.


In [6]:
vectorstore.similarity_search("headcount")

[Document(metadata={'source': 'transcript'}, page_content=".Patrick Moley, Piper Sandler: Brian, in your prepared remarks, you said that you expected a little bit of aheadcount increase in the back half of this year.I was hoping you could just elaborate on those comments and maybe help us understand how you're thinkingabout an appropriate headcount"),
 Document(metadata={'source': 'transcript'}, page_content='.Last, while we are expanding variable expenses to meet evolving market conditions, we also expect toprudently increase headcount throughout the rest of the year, primarily to support our product andinternational expansion efforts and to strengthen our product foundations and quality'),
 Document(metadata={'source': 'transcript'}, page_content='.Alesia Haas, CFO : Thanks, Brian. Our Q2 results reflect our continued revenue diversification and execution onour goal to generate positive Adjusted EBITDA in all market conditions. Total revenue was $1'),
 Document(metadata={'source': 't